- Date: 01/01/2024
- Author: Carlos Javier Navarro
- Project: EarthCul

### Code to calculate count in points, or lines or polygons

The first step is to eliminate duplicate geometries generated at the junction of the layers to avoid generating false counts.
1) To do this, first polygonize the layers so that it is a single geometry type. In the case of points, a buffer of 10 meters is set
2) Then stick only with the unique geometries.
3) Count the geometries inside of grid

In [1]:
import os
import geopandas as gpd
from shapely.geometry import Point
from shapely.ops import polygonize
from shapely.geometry import LineString

base_path = os.getcwd()

park = 'Aiguestortes'

# Input and output folders
# input_folder = os.path.join(base_path,f"../../1) Variable download/EarthCul OSM/Aiguestortes OSM\definitive variables")

input_folder = f'C:/Users/carlo/Documents/EarthCul/OSM/1) Variable download/EarthCul OSM/{park}/definitive variables'
output_folder = f'C:/Users/carlo/Documents/EarthCul/OSM/2) Derived metrics/count/Poligons/{park}'

def convert_to_polygon(geom):
    if geom.geom_type == 'Point':
        return Point(geom.x, geom.y).buffer(0.0001)
    elif geom.geom_type == 'LineString':
        polygons = list(polygonize([geom]))
        if polygons: 
            return polygons[0]  
        else:
            return None  
    else:
        return geom


# Iterate through all files in the input folder
for file_name in os.listdir(input_folder):
    if file_name.endswith('.geojson'):  
        file_path = os.path.join(input_folder, file_name)
        vector = gpd.read_file(file_path)
        vector['geometry'] = vector['geometry'].apply(convert_to_polygon)    
        output_file_path = os.path.join(output_folder, f"{os.path.splitext(file_name)[0]}.geojson")  
        vector.to_file(output_file_path, driver='GeoJSON')

c:\Users\carlo\anaconda3\Lib\site-packages\geopandas\io\file.py:612: UserWarning: You are attempting to write an empty DataFrame to file. For some drivers, this operation may fail.
  _to_file_fiona(df, filename, driver, schema, crs, mode, **kwargs)
c:\Users\carlo\anaconda3\Lib\site-packages\geopandas\io\file.py:612: UserWarning: You are attempting to write an empty DataFrame to file. For some drivers, this operation may fail.
  _to_file_fiona(df, filename, driver, schema, crs, mode, **kwargs)


2) After polygonizing select unique values 

In [ ]:
import os
import glob
import geopandas as gpd

# Path of the folder containing the GeoJSON files
input_folder = f'C:/Users/carlo/Documents/EarthCul/OSM/2) Derived metrics/count/Poligons/{park}'
output_folder = f'C:/Users/carlo/Documents/EarthCul/OSM/2) Derived metrics/count/Poligons uniques/{park}'  

# Pattern to select all GeoJSON files in folder
patron = '*.geojson'
archivos_geojson = glob.glob(os.path.join(input_folder, patron))

# Process each GeoJSON file
for archivo_geojson in archivos_geojson:
    # Load the file into a GeoDataFrame
    gdf = gpd.read_file(archivo_geojson)
    
    # Remove duplicate geometries
    gdf_sin_duplicados = gdf.drop_duplicates()
    
    # Build the output path in the new folder
    out_name = os.path.basename(archivo_geojson)
    out_path = os.path.join(output_folder, out_name)
    
    # Save the new GeoDataFrame to a new GeoJSON file
    gdf_sin_duplicados.to_file(out_path, driver='GeoJSON')
    
    print(f"Duplicate geometries removed in {archivo_geojson}. New file saved in {out_path}")

print("Completed process.")
   


In [5]:
import os

# Ruta de la carpeta que contiene los archivos GeoJSON
carpeta = f"C:/Users/carlo/Documents/EarthCul/OSM/2) Derived metrics/count/Poligons uniques/{park}/test"

# Obtener la lista de archivos GeoJSON en la carpeta
archivos_geojson = [archivo for archivo in os.listdir(carpeta) if archivo.endswith('.geojson')]

# Iterar sobre cada archivo GeoJSON y renombrarlo
for i, archivo in enumerate(archivos_geojson):
    # Construir el nuevo nombre de archivo
    nuevo_nombre = f"nuevo_nombre_{i + 1}.geojson"
    
    # Ruta completa del archivo original y nuevo
    ruta_original = os.path.join(carpeta, archivo)
    ruta_nuevo = os.path.join(carpeta, nuevo_nombre)
    
    # Renombrar el archivo
    os.rename(ruta_original, ruta_nuevo)
    
    print(f"Archivo renombrado: {archivo} -> {nuevo_nombre}")

Archivo renombrado: accommodation_Aiguestortes_3035_polygon.geojson -> nuevo_nombre_1.geojson
Archivo renombrado: amenity_bicycle_rental_Aiguestortes_3035_polygon.geojson -> nuevo_nombre_2.geojson
Archivo renombrado: amenity_community_centre_Aiguestortes_3035_polygon.geojson -> nuevo_nombre_3.geojson
Archivo renombrado: amenity_drinking_water_Aiguestortes_3035_polygon.geojson -> nuevo_nombre_4.geojson
Archivo renombrado: amenity_fountain_Aiguestortes_3035_polygon.geojson -> nuevo_nombre_5.geojson
Archivo renombrado: amenity_social_facility_Aiguestortes_3035_polygon.geojson -> nuevo_nombre_6.geojson
Archivo renombrado: barAndRestaurant_Aiguestortes_3035_polygon.geojson -> nuevo_nombre_7.geojson
Archivo renombrado: beaches_Aiguestortes_3035_polygon.geojson -> nuevo_nombre_8.geojson
Archivo renombrado: bikeRoutes_Aiguestortes_3035_polygon.geojson -> nuevo_nombre_9.geojson
Archivo renombrado: campsite_Aiguestortes_3035_polygon.geojson -> nuevo_nombre_10.geojson
Archivo renombrado: geologic


As there are some layers that are lines, it is better to count them separately.
These are
- Rivers
- Roads
- bike_routes
- hiking_trails
- railway
- streets

In [3]:
import os
import shutil
import glob

# Input and output directory
source_directory = f'C:/Users/carlo/Documents/EarthCul/OSM/2) Derived metrics/count/Poligons/{park}' 
destination_directory = f'C:/Users/carlo/Documents/EarthCul/OSM/2) Derived metrics/count/Poligons uniques/{park}' 

# Terms to search for in file names
search_terms = ["rivers", "roads", "bike", "hiking", "railway", "streets"]

# Create the destination directory if it does not exist
if not os.path.exists(destination_directory):
    os.makedirs(destination_directory)

# Get the list of files in the source directory
files_in_source = glob.glob(os.path.join(source_directory, "*.geojson"))

# Filter files based on terms
files_to_move = [file for file in files_in_source if any(term in os.path.basename(file) for term in search_terms)]

# Move the filtered files to the destination directory
for file in files_to_move:
    destination_file = os.path.join(destination_directory, os.path.basename(file))
    
    # Check if the file already exists in the destination directory
    if os.path.exists(destination_file):
        print(f"Overwriting {os.path.basename(file)} in {destination_directory}")
        os.remove(destination_file)  # Remove the existing file
    
    # Move files
    shutil.copy(file, destination_directory)

print("Files copied successfully.")


Overwriting bikeRoutes.geojson in C:/Users/carlo/Documents/Repositorio EarthCul/EarthCul/OSM/2) Derived metrics/count/Poligons uniques/Aiguestortes
Overwriting railway_rail.geojson in C:/Users/carlo/Documents/Repositorio EarthCul/EarthCul/OSM/2) Derived metrics/count/Poligons uniques/Aiguestortes
Overwriting rivers.geojson in C:/Users/carlo/Documents/Repositorio EarthCul/EarthCul/OSM/2) Derived metrics/count/Poligons uniques/Aiguestortes
Overwriting roads.geojson in C:/Users/carlo/Documents/Repositorio EarthCul/EarthCul/OSM/2) Derived metrics/count/Poligons uniques/Aiguestortes
Overwriting route_hiking.geojson in C:/Users/carlo/Documents/Repositorio EarthCul/EarthCul/OSM/2) Derived metrics/count/Poligons uniques/Aiguestortes
Overwriting streets.geojson in C:/Users/carlo/Documents/Repositorio EarthCul/EarthCul/OSM/2) Derived metrics/count/Poligons uniques/Aiguestortes
Files copied successfully.


Rename the files to simplify their name 

In [ ]:
# import os

# # Path to the folder containing the GeoJSON files
# folder_path = f"C:/Users/carlo/Documents/EarthCul/OSM/2) Derived metrics/count/Poligons uniques/{park}"

# # Dictionary mapping existing file names to desired names
# name_mapping = {
#     "accommodation.geojson": "accommodat.geojson",
#     "amenity_bicycle_rental.geojson": "bicycle_re.geojson",
#     "amenity_community_centre.geojson": "community.geojson",
#     "amenity_drinking_water.geojson": "drinking_w.geojson",
#     "amenity_fountain.geojson": "fountain.geojson",
#     "amenity_social_facility.geojson": "social_fac.geojson",
#     "barAndRestaurant.geojson": "barAndRest.geojson",
#     "beaches.geojson": "beaches.geojson",
#     "bikeRoutes.geojson": "bikeRoutes.geojson",
#     "campsite.geojson": "campsite.geojson",
#     "geological_paleontological_site.geojson": "paleontolo.geojson",
#     "glacier.geojson": "glacier.geojson",
#     "greenSpaces.geojson": "greenSpace.geojson",
#     "highway_bus_stop.geojson": "bus_stop.geojson",
#     "historic.geojson": "historic.geojson",
#     "lakes.geojson": "lakes.geojson",
#     "leisure_court.geojson": "court.geojson",
#     "parking.geojson": "parking.geojson",
#     "place_square.geojson": "square_.geojson",
#     "railway_rail.geojson": "railway.geojson",
#     "recreational.geojson": "recreation.geojson",
#     "refuges.geojson": "refuges.geojson",
#     "religious_place.geojson": "religious.geojson",
#     "rivers.geojson": "rivers.geojson",
#     "roads.geojson": "roads.geojson",
#     "route_hiking.geojson": "route_hiki.geojson",
#     "sighting_points.geojson": "sighting.geojson",
#     "ski_facilities.geojson": "ski_facili.geojson",
#     "small_town.geojson": "small_town.geojson",
#     "sport_climbing.geojson": "sport_clim.geojson",
#     "streets.geojson": "streets.geojson",
#     "tourismAttraction.geojson": "tourismAtt.geojson",
#     "volcanic.geojson": "volcanic.geojson",
#     "wifi.geojson": "wifi.geojson",
# }

# # Get the list of GeoJSON files in the folder
# geojson_files = [file for file in os.listdir(folder_path) if file.endswith('.geojson')]

# # Iterate over each GeoJSON file and rename it according to the mapping dictionary
# for file_name in geojson_files:
#     # Check if the file exists in the mapping dictionary
#     if file_name in name_mapping:
#         # Build the full path of the original file and the new name
#         original_path = os.path.join(folder_path, file_name)
#         new_name = name_mapping[file_name]
#         new_path = os.path.join(folder_path, new_name)
        
#         # Rename the file
#         os.rename(original_path, new_path)
        
#         print(f"File renamed: {file_name} -> {new_name}")
#     else:
#         print(f"No mapped file name found for: {file_name}")



3) Make counts of all variables listed in a folder, this process take time (more than 30 minutes) depends the size of the study area

In [4]:
import os
import json
import geopandas as gpd

# Path to the shapefile
shapefile_path = f'C:/Users/carlo/Documents/EarthCul/OSM/Grids/{park}/grid_wgs84_atrib.shp'

# Read the shapefile using geopandas
grid = gpd.read_file(shapefile_path)

# Path to the folder containing the GeoJSON files
geojson_folder = f'C:/Users/carlo/Documents/EarthCul/OSM/2) Derived metrics/count/Poligons uniques/{park}'

# List all files in the folder with .geojson extension
geojson_files = [file for file in os.listdir(geojson_folder) if file.endswith('.geojson')]

# Iterate over each GeoJSON file
for geojson_file in geojson_files:
    # Construct the full path of the GeoJSON file
    geojson_path = os.path.join(geojson_folder, geojson_file)
    
    # Read the GeoJSON
    with open(geojson_path, encoding='utf-8') as f:
        point_data = json.load(f)
    
    # Convert the GeoJSON to GeoDataFrame
    gdf = gpd.GeoDataFrame.from_features(point_data['features'])
    
    if gdf.empty:
        pass
    else:
        # Create a new column in the grid GeoDataFrame to store point counts
        column_name = f'{geojson_file}'  # Use part of the file name as the column name f'{geojson_file[:-13]}'
        grid[column_name] = 0

        # Iterate over each grid cell
        for idx, cell in grid.iterrows():
            # Filter points that are within the current cell 
            # points_in_cell = gdf[gdf.geometry.within(cell['geometry'])]

            points_in_cell = gdf[gdf.geometry.intersects(cell['geometry'])]

            # Get the number of points in the cell and update the corresponding column
            grid.at[idx, column_name] = len(points_in_cell)

# Save the result to a new shapefile
grid.to_file(f'C:/Users/carlo/Documents/EarthCul/OSM/2) Derived metrics/count/results/{park}/grid_wgs84_atrib_count.shp', driver='ESRI Shapefile')



C:\Users\carlo\AppData\Local\Temp\ipykernel_9516\3258722796.py:47: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  grid.to_file(f'C:/Users/carlo/Documents/EarthCul/OSM/2) Derived metrics/count/results/{park}/grid_wgs84_atrib_count.shp', driver='ESRI Shapefile')


To reproject the grid with the count values

In [5]:
gdf = gpd.read_file(f'C:/Users/carlo/Documents/EarthCul/OSM/2) Derived metrics/count/results/{park}/grid_wgs84_atrib_count.shp')
gdf = gdf.to_crs('EPSG:3035')
# Save the GeoDataFrame to a new GeoJSON file
gdf.to_file(f'C:/Users/carlo/Documents/EarthCul/OSM/2) Derived metrics/count/results/{park}/grid_3035_count.shp', driver='ESRI Shapefile')

Finally rasterize the columns where from the grid with the values of the counts

In [6]:
from tqdm import tqdm
import os
import geopandas as gpd
import rasterio
from rasterio import features
# Grid and base raster
Grid = f'C:/Users/carlo/Documents/EarthCul/OSM/2) Derived metrics/count/results/{park}/grid_3035_count.shp'

Base_raster= f"C:/Users/carlo/Documents/EarthCul/OSM/2) Derived metrics/BaseLayers/{park}/Slope.tif" 
              
# Output folder
output_folder = f"C:/Users/carlo/Documents/EarthCul/OSM/2) Derived metrics/count/results/{park}"  

gdf = gpd.read_file(Grid)

with rasterio.open(Base_raster) as src:
    profile = src.profile  

    # Loop through columns to rasterize 
    columns_to_process = gdf.columns.tolist()[3:]
    for column in tqdm(columns_to_process, desc='Rasterizing Columns'):
        values = gdf[column].values
        
        # Rasterize the values
        rasterized = features.rasterize(
            zip(gdf.geometry, values),
            out_shape=(profile['height'], profile['width']),
            transform=profile['transform'],
            fill=-999.99,  
            all_touched=True,  
            default_value=0,  
            dtype=profile['dtype']  
        )
 
        output_path = os.path.join(output_folder, f'{column}.tif')
        
        # Save the outputs
        with rasterio.open(output_path, 'w', **profile) as dst:
            dst.write(rasterized, 1)


Rasterizing Columns:  97%|█████████▋| 31/32 [11:32<00:22, 22.34s/it]


TypeError: '<=' not supported between instances of 'Polygon' and 'Polygon'